In [538]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import time
import requests
from io import StringIO
import os

In [539]:
def fetch_data_from_web(base_url, limit=1000):
    offset = 0
    all_data = []

    def get_data(base_url, offset):
        params = {
            "$limit": limit,
            "$offset": offset
        }
        response = requests.get(base_url, params=params)
        if response.status_code == 200:
            print(f"Data retrieved from {base_url} with offset: {offset}")
            return pd.read_csv(StringIO(response.text))
        else:
            print(f"Failed to retrieve data from {base_url} at offset {offset}")
            return pd.DataFrame()

    while True:
        df = get_data(base_url, offset)
        if df.empty:
            print(f"No more data to retrieve from {base_url}.")
            break
        all_data.append(df)
        offset += limit
        time.sleep(1)  # avoid rate limit

    final_df = pd.concat(all_data, ignore_index=True)
    return final_df

In [540]:
def robust_datetime_parsing(df, column_name):
    # Save original column
    original = df[column_name].copy()

    # First pass
    df[column_name] = pd.to_datetime(original, utc=True, errors='coerce')

    # Second pass only where failed
    mask_failed = df[column_name].isna()
    df.loc[mask_failed, column_name] = pd.to_datetime(
        original[mask_failed],
        format='%Y-%m-%dT%H:%M:%S',
        utc=True,
        errors='coerce'
    )


In [541]:
def prepare_dataframes(df_intake, df_outcome):
    # Step 1: Clean animal_ids
    df_intake['animal_id'] = df_intake['animal_id'].str.strip().str.lower()
    df_outcome['animal_id'] = df_outcome['animal_id'].str.strip().str.lower()

    # Step 2: Parse datetimes properly
    robust_datetime_parsing(df_intake, 'datetime')
    robust_datetime_parsing(df_outcome, 'datetime')

    # Step 3 onwards = SAME as your previous code...
    latest_intake = df_intake.sort_values(by='datetime', ascending=False).drop_duplicates(subset='animal_id')
    latest_outcome = df_outcome.sort_values(by='datetime', ascending=False).drop_duplicates(subset='animal_id')

    common_animal_ids = set(latest_intake['animal_id']).intersection(set(latest_outcome['animal_id']))
    latest_intake = latest_intake[latest_intake['animal_id'].isin(common_animal_ids)]
    latest_outcome = latest_outcome[latest_outcome['animal_id'].isin(common_animal_ids)]

    latest_intake = latest_intake.rename(columns={
        'name': 'name_intake',
        'datetime': 'datetime_intake',
        'found_location': 'found_location',
        'intake_type': 'intake_type',
        'intake_condition': 'intake_condition',
        'animal_type': 'animal_type_intake',
        'sex_upon_intake': 'sex_upon_intake',
        'age_upon_intake': 'age_upon_intake',
        'breed': 'breed_intake',
        'color': 'color_intake'
    })

    latest_outcome = latest_outcome.rename(columns={
        'name': 'name_outcome',
        'datetime': 'datetime_outcome',
        'monthyear': 'monthyear_outcome',
        'date_of_birth': 'date_of_birth_outcome',
        'outcome_type': 'outcome_type',
        'outcome_subtype': 'outcome_subtype',
        'animal_type': 'animal_type_outcome',
        'sex_upon_outcome': 'sex_upon_outcome',
        'age_upon_outcome': 'age_upon_outcome',
        'breed': 'breed_outcome',
        'color': 'color_outcome'
    })

    animal_df = pd.merge(latest_intake, latest_outcome, on='animal_id', how='inner')

    if 'outcome_subtype' in animal_df.columns:
        animal_df.drop(columns=['outcome_subtype'], inplace=True)

    print(f"Intake entries (after filtering): {len(latest_intake)}")
    print(f"Outcome entries (after filtering): {len(latest_outcome)}")
    print(f"Merged entries (animal_df): {len(animal_df)}")

    print("\nSample animal_id from intake:")
    print(latest_intake['animal_id'].dropna().unique()[:10])

    print("\nSample animal_id from outcome:")
    print(latest_outcome['animal_id'].dropna().unique()[:10])

    return animal_df


In [542]:
def feature_engineering(animal_df):
    print(f"Start feature_engineering(): {animal_df.shape[0]} rows")

    null_counts = animal_df.isnull().sum()
    null_percentages = (null_counts / len(animal_df)) * 100
    null_info = pd.DataFrame({'Null Counts': null_counts, 'Null Percentage': null_percentages})
    print("\nNulls BEFORE any cleaning:")
    print(null_info.sort_values('Null Percentage', ascending=False))


    animal_df['has_name'] = animal_df['name_intake'].notna().astype(int)

    # Drop rows missing critical dates
    essential_cols = ['datetime_intake', 'datetime_outcome']
    dob_col = 'date_of_birth_outcome' if 'date_of_birth_outcome' in animal_df.columns else 'date_of_birth'
    essential_cols.append(dob_col)
    animal_df.dropna(subset=essential_cols, inplace=True)

    print(f"After dropna(): {animal_df.shape[0]} rows")

    animal_df['outcome_group'] = animal_df['outcome_type'].apply(group_outcome)
    animal_df = animal_df[animal_df['outcome_group'] != 'Other'].copy()
    print(f"After outcome group filtering: {animal_df.shape[0]} rows")

    date_cols = ['datetime_intake', 'datetime_outcome']
    if 'date_of_birth_outcome' in animal_df.columns:
        date_cols.append('date_of_birth_outcome')
    elif 'date_of_birth' in animal_df.columns:
        date_cols.append('date_of_birth')

    for col in date_cols:
        animal_df[col] = pd.to_datetime(animal_df[col], errors='coerce')
        if animal_df[col].dt.tz is not None:
            animal_df[col] = animal_df[col].dt.tz_convert(None)

    dob_col = 'date_of_birth_outcome' if 'date_of_birth_outcome' in animal_df.columns else 'date_of_birth'

    animal_df['age_days_intake'] = (animal_df['datetime_intake'] - animal_df[dob_col]).dt.days
    animal_df['age_days_outcome'] = (animal_df['datetime_outcome'] - animal_df[dob_col]).dt.days
    animal_df = animal_df[(animal_df['age_days_intake'] > 0) & (animal_df['age_days_outcome'] > 0)].copy()
    print(f"After age_days filter: {animal_df.shape[0]} rows")

    animal_df['age_group_intake'] = animal_df['age_days_intake'].apply(classify_age)
    animal_df['age_group_outcome'] = animal_df['age_days_outcome'].apply(classify_age)

    animal_df['los_at_shelter'] = (animal_df['datetime_outcome'] - animal_df['datetime_intake']).dt.days
    animal_df = animal_df[animal_df['los_at_shelter'] > 0].copy()
    print(f"After los_at_shelter filter: {animal_df.shape[0]} rows")

    animal_df['month_of_outcome'] = animal_df['datetime_outcome'].dt.strftime('%B')
    animal_df['reproductive_status'] = animal_df['sex_upon_outcome'].apply(simplify_sex)
    animal_df['is_fixed'] = animal_df['reproductive_status'].apply(lambda x: 1 if x == 'Fixed' else 0)

    animal_df['breed_type'] = animal_df['breed_intake'].apply(lambda x: 'Mix' if 'Mix' in str(x) else 'Pure')
    animal_df['color_group'] = animal_df['color_intake'].apply(simplify_color)
    animal_df['intake_condition_group'] = animal_df['intake_condition'].apply(group_intake_condition)

    animal_df.rename(columns={'animal_type_intake': 'animal_type'}, inplace=True)
    animal_df.drop(columns=['animal_type_outcome'], inplace=True)

    print(f"End of feature_engineering(): {animal_df.shape[0]} rows")
    
    return animal_df


In [543]:
def final_cleaning(animal_df):
    final_drop_cols = [
        'animal_id', 'datetime_intake', 'datetime_outcome', 'datetime2', 'found_location',
        'intake_type', 'intake_condition', 'breed_intake', 'color_intake', 'monthyear_outcome',
        'date_of_birth_outcome', 'outcome_type', 'sex_upon_intake', 'sex_upon_outcome',
        'age_upon_intake', 'age_upon_outcome', 'breed_outcome', 'color_outcome',
        'reproductive_status', 'age_group_outcome'
    ]
    animal_df.drop(columns=[col for col in final_drop_cols if col in animal_df.columns], inplace=True)
    animal_df.drop(columns=['age_days_intake'], inplace=True, errors='ignore')
    return animal_df

In [544]:
def save_clean_data(animal_df, filename='animal_df.csv'):
    animal_df.to_csv(filename, index=False)


In [545]:
def group_outcome(outcome):
    if outcome in ['Adoption', 'Return to Owner', 'Rto-Adopt']:
        return 'Positive'
    elif outcome in ['Transfer', 'Relocate']:
        return 'Neutral'
    elif outcome in ['Euthanasia', 'Died', 'Disposal', 'Lost', 'Missing', 'Stolen']:
        return 'Negative'
    else:
        return 'Other'

In [546]:
def classify_age(days):
    if days < 365:
        return 'Puppy/Kitten'
    elif days < 1095:
        return 'Young Adult'
    elif days < 2555:
        return 'Adult'
    else:
        return 'Senior'

In [547]:
def simplify_sex(sex):
    if sex in ['Neutered Male', 'Spayed Female']:
        return 'Fixed'
    elif sex in ['Intact Male', 'Intact Female']:
        return 'Intact'
    else:
        return 'Unknown'

In [548]:
def simplify_color(color):
    color = str(color).lower()
    groups = {
        'Black': ['black', 'ebony', 'charcoal'],
        'White': ['white', 'cream', 'ivory', 'pearl', 'platinum'],
        'Brown': ['brown', 'brindle', 'chocolate', 'mocha', 'mahogany'],
        'Grey': ['gray', 'grey', 'blue', 'silver', 'slate', 'pewter', 'ash'],
        'Gold': ['gold', 'tan', 'yellow', 'fawn', 'buff', 'sandy'],
        'Red': ['red', 'ginger', 'orange', 'copper', 'auburn'],
        'Cream': ['cream', 'beige', 'off-white'],
    }
    for group, keywords in groups.items():
        if any(word in color for word in keywords):
            return group
    if 'tabby' in color:
        return 'Tabby'
    if 'merle' in color:
        return 'Merle'
    if 'tricolor' in color:
        return 'Tricolor'
    if any(word in color for word in ['spotted', 'speckled', 'ticked', 'roan', 'freckled']):
        return 'Spotted'
    if '/' in color or 'and' in color:
        return 'Multicolor'
    return 'Other'


In [549]:
def group_intake_condition(condition):
    condition = str(condition).lower()
    if condition == 'normal': return 'Healthy/Normal'
    if condition in ['injured', 'neurologic', 'med attn', 'med urgent']: return 'Injured'
    if condition in ['sick', 'medical', 'parvo', 'panleuk', 'congenital']: return 'Sick/Medical'
    if condition in ['behavior', 'feral']: return 'Behavioral Issues'
    if condition in ['neonatal', 'nursing', 'aged', 'pregnant']: return 'Life Stage/Developmental'
    return 'Other/Unknown'


In [550]:
def main():
    # Check if we already have merged file
    if os.path.exists('merged_animal_data_raw.csv'):
        print("Found existing merged file: merged_animal_data_raw.csv")
        animal_df = pd.read_csv('merged_animal_data_raw.csv')
        print(f"Loaded merged dataset shape: {animal_df.shape}")

    else:
        # Step 1: Check if raw datasets exist
        if os.path.exists('intake_raw.csv') and os.path.exists('outcome_raw.csv'):
            print("Found existing raw datasets: intake_raw.csv and outcome_raw.csv")
            intake_df = pd.read_csv('intake_raw.csv')
            outcome_df = pd.read_csv('outcome_raw.csv')

        else:
            # Step 2: Otherwise fetch from API
            print("Fetching Outcome Data...")
            outcome_df = fetch_data_from_web("https://data.austintexas.gov/resource/9t4d-g238.csv")
            print(f"Outcome dataset shape: {outcome_df.shape}")

            print("Fetching Intake Data...")
            intake_df = fetch_data_from_web("https://data.austintexas.gov/resource/wter-evkm.csv")
            print(f"Intake dataset shape: {intake_df.shape}")

            # Step 3: Save raw copies
            intake_df.to_csv('intake_raw.csv', index=False)
            outcome_df.to_csv('outcome_raw.csv', index=False)
            print("Saved raw datasets to intake_raw.csv and outcome_raw.csv")

        # Step 4: Merge intake and outcome
        print("Merging datasets...")
        animal_df = prepare_dataframes(intake_df, outcome_df)

        # Step 5: Save merged
        animal_df.to_csv('merged_animal_data_raw.csv', index=False)
        print("Saved merged dataset to merged_animal_data_raw.csv")

    print(f"\nLoaded dataset shape: {animal_df.shape}")

    print("Applying feature engineering...")
    animal_df = feature_engineering(animal_df)

    print("Final cleaning...")
    animal_df = final_cleaning(animal_df)

    print("Saving cleaned data...")
    save_clean_data(animal_df)

    print("\nDone! Here's a preview:")
    display(animal_df.head())

In [551]:
if __name__ == "__main__":
    main()

Found existing merged file: merged_animal_data_raw.csv
Loaded merged dataset shape: (155231, 22)

Loaded dataset shape: (155231, 22)
Applying feature engineering...
Start feature_engineering(): 155231 rows

Nulls BEFORE any cleaning:
                       Null Counts  Null Percentage
name_outcome                 49174        31.677951
name_intake                  49174        31.677951
outcome_type                    37         0.023835
age_upon_outcome                15         0.009663
sex_upon_outcome                 2         0.001288
sex_upon_intake                  1         0.000644
animal_id                        0         0.000000
date_of_birth_outcome            0         0.000000
breed_outcome                    0         0.000000
animal_type_outcome              0         0.000000
monthyear_outcome                0         0.000000
datetime_outcome                 0         0.000000
color_intake                     0         0.000000
breed_intake                     0    

,name_intake,animal_type,name_outcome,has_name,outcome_group,age_days_outcome,age_group_intake,los_at_shelter,month_of_outcome,is_fixed,breed_type,color_group,intake_condition_group
15,NaN,Cat,NaN,0,Negative,366,Young Adult,1,April,0,Pure,Red,Healthy/Normal
16,Mari,Cat,Mari,1,Positive,367,Young Adult,1,April,1,Pure,Other,Healthy/Normal
23,NaN,Other,NaN,0,Negative,732,Young Adult,1,April,0,Pure,Brown,Healthy/Normal
25,Akxel,Dog,Akxel,1,Positive,788,Young Adult,1,April,1,Pure,Black,Healthy/Normal
26,Hashbrown,Cat,Hashbrown,1,Positive,245,Puppy/Kitten,2,April,1,Pure,Black,Healthy/Normal


In [552]:
# 1. Load the two raw datasets
intake_df = pd.read_csv('intake_raw.csv')
outcome_df = pd.read_csv('outcome_raw.csv')

print(f"Intake shape: {intake_df.shape}")
print(f"Outcome shape: {outcome_df.shape}")

# 2. Check missing values
print("\nMissing values in intake dataset:")
display(intake_df.isnull().sum())

print("\nMissing values in outcome dataset:")
display(outcome_df.isnull().sum())

# 3. Focus on datetime column
print("\nPercentage of missing 'datetime' in outcome dataset:")
missing_datetime_outcome = outcome_df['datetime'].isnull().mean() * 100
print(f"Missing datetime in outcome: {missing_datetime_outcome:.2f}%")

# 4. Check rows where datetime is missing
print("\nExamples of outcome rows where 'datetime' is missing:")
display(outcome_df[outcome_df['datetime'].isnull()].head(10))

# 5. Compare with intake
print("\nExamples of intake rows where 'datetime' is missing:")
display(intake_df[intake_df['datetime'].isnull()].head(10))

# 6. Check if rows with missing datetime still have other important columns filled
print("\nOutcome rows with missing datetime but non-null 'animal_id':")
display(outcome_df[(outcome_df['datetime'].isnull()) & (outcome_df['animal_id'].notnull())].head(10))


Intake shape: (173544, 12)
Outcome shape: (173567, 12)

Missing values in intake dataset:


animal_id               0
name                49828
datetime                0
datetime2               0
found_location          0
intake_type             0
intake_condition        0
animal_type             0
sex_upon_intake         1
age_upon_intake         0
breed                   0
color                   0
dtype: int64


Missing values in outcome dataset:


animal_id               0
date_of_birth           0
name                49700
datetime                0
monthyear               0
outcome_type           46
outcome_subtype     94019
animal_type             0
sex_upon_outcome        2
age_upon_outcome       15
breed                   0
color                   0
dtype: int64


Percentage of missing 'datetime' in outcome dataset:
Missing datetime in outcome: 0.00%

Examples of outcome rows where 'datetime' is missing:


,animal_id,date_of_birth,name,datetime,monthyear,outcome_type,outcome_subtype,animal_type,sex_upon_outcome,age_upon_outcome,breed,color



Examples of intake rows where 'datetime' is missing:


,animal_id,name,datetime,datetime2,found_location,intake_type,intake_condition,animal_type,sex_upon_intake,age_upon_intake,breed,color



Outcome rows with missing datetime but non-null 'animal_id':


,animal_id,date_of_birth,name,datetime,monthyear,outcome_type,outcome_subtype,animal_type,sex_upon_outcome,age_upon_outcome,breed,color


In [553]:
import pandas as pd

# 1. Load the raw datasets
intake_df = pd.read_csv('intake_raw.csv')
outcome_df = pd.read_csv('outcome_raw.csv')

# 2. Explore NULLS BEFORE MERGE (in raw outcome)
null_counts_outcome = outcome_df.isnull().sum()
null_percentages_outcome = (null_counts_outcome / len(outcome_df)) * 100

null_info_outcome = pd.DataFrame({
    'Null Counts': null_counts_outcome,
    'Null Percentage': null_percentages_outcome
}).sort_values(by='Null Counts', ascending=False)

print("\n🧹 Null counts BEFORE merge in OUTCOME dataframe:")
display(null_info_outcome)

# 3. Then, prepare dataframes (with your function)
animal_df = prepare_dataframes(intake_df, outcome_df)

# 4. Explore NULLS AFTER MERGE
null_counts_after_merge = animal_df.isnull().sum()
null_percentages_after_merge = (null_counts_after_merge / len(animal_df)) * 100

null_info_after_merge = pd.DataFrame({
    'Null Counts': null_counts_after_merge,
    'Null Percentage': null_percentages_after_merge
}).sort_values(by='Null Counts', ascending=False)

print("\n📦 Null counts AFTER merge in merged ANIMAL dataframe:")
display(null_info_after_merge)



🧹 Null counts BEFORE merge in OUTCOME dataframe:


,Null Counts,Null Percentage
outcome_subtype,94019,54.168707
name,49700,28.634475
outcome_type,46,0.026503
age_upon_outcome,15,0.008642
sex_upon_outcome,2,0.001152
animal_id,0,0.000000
date_of_birth,0,0.000000
datetime,0,0.000000
monthyear,0,0.000000
animal_type,0,0.000000


Intake entries (after filtering): 155231
Outcome entries (after filtering): 155231
Merged entries (animal_df): 155231

Sample animal_id from intake:
['a929258' 'a929257' 'a929256' 'a929259' 'a929260' 'a929252' 'a928219'
 'a929231' 'a893642' 'a929225']

Sample animal_id from outcome:
['a926413' 'a917294' 'a927291' 'a928531' 'a929020' 'a923553' 'a923430'
 'a928235' 'a928236' 'a929260']

📦 Null counts AFTER merge in merged ANIMAL dataframe:


,Null Counts,Null Percentage
name_outcome,49174,31.677951
name_intake,49174,31.677951
outcome_type,37,0.023835
age_upon_outcome,15,0.009663
sex_upon_outcome,2,0.001288
sex_upon_intake,1,0.000644
animal_id,0,0.000000
date_of_birth_outcome,0,0.000000
breed_outcome,0,0.000000
animal_type_outcome,0,0.000000


In [554]:
def prepare_dataframes_from_csv(intake_csv_path, outcome_csv_path):
    print("Loading raw data from CSVs...")
    
    # 1. Load from your stored raw CSVs
    df_intake = pd.read_csv(intake_csv_path)
    df_outcome = pd.read_csv(outcome_csv_path)
    
    print(f"Intake shape: {df_intake.shape}")
    print(f"Outcome shape: {df_outcome.shape}")

    # 2. Parse datetimey
    outcome_df['datetime'] = pd.to_datetime(outcome_df['datetime'], errors='coerce')

    missing_datetime_mask = outcome_df['datetime'].isna()

    # Parse monthyear into a temporary datetime (first day of the month)
    outcome_df.loc[missing_datetime_mask, 'datetime'] = pd.to_datetime(
        outcome_df.loc[missing_datetime_mask, 'monthyear'],
        format='%m-%Y',
        errors='coerce'
    )


    # 3. Keep the latest intake and latest outcome per animal_id
    latest_intake = df_intake.sort_values(by='datetime', ascending=False).drop_duplicates(subset='animal_id', keep='first')
    final_outcome = df_outcome.sort_values(by='datetime', ascending=False).drop_duplicates(subset='animal_id', keep='first')

    # 4. Clean animal_id (strip + lowercase)
    latest_intake['animal_id'] = latest_intake['animal_id'].str.strip().str.lower()
    final_outcome['animal_id'] = final_outcome['animal_id'].str.strip().str.lower()

    common_ids = set(latest_intake['animal_id']).intersection(set(final_outcome['animal_id']))

    print(f"Number of common animal_ids: {len(common_ids)}")

    # 5. Rename columns
    latest_intake = latest_intake.rename(columns={
        'name': 'name_intake',
        'datetime': 'datetime_intake',
        'found_location': 'found_location',
        'intake_type': 'intake_type',
        'intake_condition': 'intake_condition',
        'animal_type': 'animal_type_intake',
        'sex_upon_intake': 'sex_upon_intake',
        'age_upon_intake': 'age_upon_intake',
        'breed': 'breed_intake',
        'color': 'color_intake'
    })

    final_outcome = final_outcome.rename(columns={
        'name': 'name_outcome',
        'datetime': 'datetime_outcome',
        'monthyear': 'monthyear_outcome',
        'date_of_birth': 'date_of_birth_outcome',
        'outcome_type': 'outcome_type',
        'outcome_subtype': 'outcome_subtype',
        'animal_type': 'animal_type_outcome',
        'sex_upon_outcome': 'sex_upon_outcome',
        'age_upon_outcome': 'age_upon_outcome',
        'breed': 'breed_outcome',
        'color': 'color_outcome'
    })
    # BEFORE merging, in prepare_dataframes()
    if 'datetime2' in latest_intake.columns:
        latest_intake.drop(columns=['datetime2'], inplace=True)

    # 6. Merge
    animal_df = pd.merge(latest_intake, final_outcome, on='animal_id', how='inner')

    print(f"\nAnimal DataFrame Shape (Latest Intake + Outcome merged): {animal_df.shape}")
    print(animal_df.head())

    # 7. Optional: Check null info (for exploration)
    null_counts = animal_df.isnull().sum()
    null_percentages = (null_counts / len(animal_df)) * 100
    null_info = pd.DataFrame({'Null Counts': null_counts, 'Null Percentage': null_percentages})
    print("\nNull info after merging:")
    print(null_info)

    return animal_df


In [555]:
def main():
    # Paths to your stored CSVs
    intake_csv_path = "intake_raw.csv"
    outcome_csv_path = "outcome_raw.csv"

    # Prepare the animal dataframe
    animal_df = prepare_dataframes_from_csv(intake_csv_path, outcome_csv_path)

if __name__ == "__main__":
    main()


Loading raw data from CSVs...
Intake shape: (173544, 12)
Outcome shape: (173567, 12)


C:\Users\cleme\AppData\Local\Temp\ipykernel_44628\4520204.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '<DatetimeArray>
[]
Length: 0, dtype: datetime64[ns]' has dtype incompatible with datetime64[ns, UTC], please explicitly cast to a compatible dtype first.
  outcome_df.loc[missing_datetime_mask, 'datetime'] = pd.to_datetime(


Number of common animal_ids: 155231

Animal DataFrame Shape (Latest Intake + Outcome merged): (155231, 22)
  animal_id name_intake          datetime_intake  \
0   a929258         NaN  2025-04-27T12:46:00.000   
1   a929257         NaN  2025-04-27T12:46:00.000   
2   a929256         NaN  2025-04-27T12:46:00.000   
3   a929259         NaN  2025-04-27T12:46:00.000   
4   a929260         NaN  2025-04-27T12:46:00.000   

                    found_location intake_type intake_condition  \
0  13012 Moore Road in Austin (TX)       Stray         Neonatal   
1  13012 Moore Road in Austin (TX)       Stray         Neonatal   
2  13012 Moore Road in Austin (TX)       Stray         Neonatal   
3  13012 Moore Road in Austin (TX)       Stray         Neonatal   
4  13012 Moore Road in Austin (TX)       Stray         Neonatal   

  animal_type_intake sex_upon_intake age_upon_intake        breed_intake  ...  \
0                Cat   Intact Female         1 weeks  Domestic Shorthair  ...   
1              

In [556]:
outcome_df = pd.read_csv('outcome_raw.csv')  # if not loaded yet
outcome_df['datetime'] = pd.to_datetime(outcome_df['datetime'], errors='coerce')

# Now count the number of unique datetimes
num_unique_datetimes = outcome_df['datetime'].nunique()

print(f"Number of unique datetime values in outcome_df: {num_unique_datetimes}")

# Count the occurrences of each datetime value
datetime_counts = outcome_df['datetime'].value_counts()

# Display the most common ones
print(datetime_counts.head(300))


Number of unique datetime values in outcome_df: 1271
datetime
2016-04-18 00:00:00-05:00    39
2015-08-11 00:00:00-05:00    25
2017-10-17 00:00:00-05:00    25
2024-06-24 00:00:00-05:00    22
2019-06-27 00:00:00-05:00    22
                             ..
2023-02-18 00:00:00-05:00     3
2023-02-16 00:00:00-05:00     3
2016-12-16 00:00:00-05:00     3
2017-01-30 00:00:00-05:00     3
2017-04-24 00:00:00-05:00     3
Name: count, Length: 300, dtype: int64


In [557]:
outcome_df = pd.read_csv('outcome_raw.csv')

# First parse: automatic UTC
outcome_df['parsed_datetime'] = pd.to_datetime(outcome_df['datetime'], utc=True, errors='coerce')

# Second parse: manual format for those that failed
mask_failed = outcome_df['parsed_datetime'].isna()
outcome_df.loc[mask_failed, 'parsed_datetime'] = pd.to_datetime(
    outcome_df.loc[mask_failed, 'datetime'],
    format='%Y-%m-%dT%H:%M:%S',
    utc=True,
    errors='coerce'
)

# Mask where parsing worked (CHECK ONLY parsed_datetime!)
parsed_ok_mask = outcome_df['parsed_datetime'].apply(lambda x: isinstance(x, pd.Timestamp))

# Good parses
good_formats = outcome_df.loc[parsed_ok_mask, 'datetime']

# Bad parses
bad_formats = outcome_df.loc[~parsed_ok_mask, 'datetime']

print(f"✅ Number of successful datetime parses: {good_formats.shape[0]}")
print("\n✅ Examples of good formats:")
print(good_formats.unique()[:10])

print("\n" + "="*50 + "\n")

print(f"❌ Number of failed datetime parses: {bad_formats.shape[0]}")
print("\n❌ Examples of bad formats:")
print(bad_formats.unique()[:10])


✅ Number of successful datetime parses: 173567

✅ Examples of good formats:
['2013-12-02T00:00:00-05:00' '2014-02-22T00:00:00-05:00'
 '2014-04-07T00:00:00-05:00' '2014-05-16T00:00:00-05:00'
 '2014-06-10T00:00:00-05:00' '2014-07-11T00:00:00-05:00'
 '2014-07-12T00:00:00-05:00' '2014-07-14T00:00:00-05:00'
 '2014-08-20T00:00:00-05:00' '2014-09-16T00:00:00-05:00']


❌ Number of failed datetime parses: 0

❌ Examples of bad formats:
[]


In [558]:
outcome_df = pd.read_csv('intake_raw.csv')

# First parse: automatic UTC
outcome_df['parsed_datetime'] = pd.to_datetime(outcome_df['datetime'], utc=True, errors='coerce')

# Second parse: manual format for those that failed
mask_failed = outcome_df['parsed_datetime'].isna()
outcome_df.loc[mask_failed, 'parsed_datetime'] = pd.to_datetime(
    outcome_df.loc[mask_failed, 'datetime'],
    format='%Y-%m-%dT%H:%M:%S',
    utc=True,
    errors='coerce'
)

# Mask where parsing worked (CHECK ONLY parsed_datetime!)
parsed_ok_mask = outcome_df['parsed_datetime'].apply(lambda x: isinstance(x, pd.Timestamp))

# Good parses
good_formats = outcome_df.loc[parsed_ok_mask, 'datetime']

# Bad parses
bad_formats = outcome_df.loc[~parsed_ok_mask, 'datetime']

print(f"✅ Number of successful datetime parses: {good_formats.shape[0]}")
print("\n✅ Examples of good formats:")
print(good_formats.unique()[:10])

print("\n" + "="*50 + "\n")

print(f"❌ Number of failed datetime parses: {bad_formats.shape[0]}")
print("\n❌ Examples of bad formats:")
print(bad_formats.unique()[:10])


✅ Number of successful datetime parses: 173544

✅ Examples of good formats:
['2013-10-01T07:51:00.000' '2013-10-01T08:33:00.000'
 '2013-10-01T08:53:00.000' '2013-10-01T09:33:00.000'
 '2013-10-01T10:37:00.000' '2013-10-01T10:59:00.000'
 '2013-10-01T11:01:00.000' '2013-10-01T11:09:00.000'
 '2013-10-01T11:15:00.000' '2013-10-01T11:21:00.000']


❌ Number of failed datetime parses: 0

❌ Examples of bad formats:
[]
